In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
import datetime as dt
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import json
from selenium import webdriver
import time

In [213]:
city = "Washington DC"

In [214]:
if len(city.split()) > 1:
    urlcity =city.split()[0].lower()+"%20"+city.split()[1].lower()
urlpage = "https://www.google.com/search?q="+str(urlcity)+"%20nightclubs"

In [215]:
driver = webdriver.Firefox()
# get web page
driver.get(urlpage)

In [216]:
ncdf = pd.DataFrame(columns=['Rating', 'Cost', 'Num of Reviews', 'Type', 'City', 'Opening Time', 'Description'])
row = 0

In [217]:
driver.find_elements_by_class_name('DLOTif')[0].click()

In [218]:
#time.sleep(30)
results = driver.find_elements_by_class_name('dbg0pd')
titles = [re.search("\>(.*)\<", x.get_attribute('innerHTML')).group(1).replace("<wbr>", "").replace("&amp;", "&") for x in results]
titledf = pd.DataFrame(titles, columns=["Name"]) 
titledf
#driver.quit()

,Name
0,DC9 Nightclub
1,Bliss Nightclub
2,Decades DC
3,Soundcheck
4,Ultrabar
5,Heist Night Club
6,U Street Music Hall
7,Eighteenth Street Lounge
8,Eden Lounge DC
9,Club Timehri


In [219]:
results = driver.find_elements_by_class_name('lqhpac')
for result in results:
    listing = []
    rating, timing, costtyp, address, desc, cost, typ, numreviews = "", "", "", "", "", "", "", ""
    for dataline in result.text.split("\n"):
        if dataline.rfind(".") == 1 and rating == "":
            rating = dataline
        elif dataline.find("PM") != -1 and len(dataline) <=23 and timing == "":
            timing = dataline
            for attr in timing.split("⋅"):
                if attr.find("PM") != -1:
                    timing = dataline.strip()
        elif (dataline.rfind("·") != -1) and (dataline.rfind("$") != -1 or dataline == result.text.split("\n")[1]) and (costtyp == ""):
            costtyp = dataline
            for attribute in costtyp.split("·"):
                if attribute.find("$") != -1:
                    cost = attribute.strip()
                elif attribute.find(")") != -1:
                    numreviews = attribute.replace("(","").replace(")","").strip()
                else:
                    typ = attribute.strip()
        elif (len(dataline) >= 35 and dataline == result.text.split("\n")[len(result.text.split("\n"))-1]) or (dataline == result.text.split("\n")[len(result.text.split("\n"))-1]) or (address != ""):
            desc = desc + dataline
        else:
            if(address == ""):
                address = dataline
    #print(rating)
    #print(cost)
    #print(numreviews)
    #print(typ)
    #print(address)
    #print(timing)
    #print(desc)
    listing.append(rating)
    listing.append(cost)
    listing.append(numreviews)
    listing.append(typ)
    listing.append(address)
    listing.append(timing)
    listing.append(desc)
    ncdf.loc[row] = listing
    row += 1
ncdf = pd.concat([titledf, ncdf], axis = 1)
ncdf

,Name,Rating,Cost,Num of Reviews,Type,City,Opening Time,Description
0,DC9 Nightclub,4.3,$,346,Night club,1940 9th St NW,Closed ⋅ Opens 5PM,Tri-level hipster hangout with snug basement b...
1,Bliss Nightclub,2.9,$$,228,Night club,2122 24th Pl NE,Closed ⋅ Opens 10PM Fri,"Sprawling dance club featuring live hip-hop, r..."
2,Decades DC,3.3,$$,237,Night club,1219 Connecticut Ave NW,Closed ⋅ Opens 10PM Thu,"Retro-themed, 3-story nightclub featuring '80s..."
3,Soundcheck,3.8,$$,166,Night club,1420 K St NW,Closed ⋅ Opens 10PM Wed,Their website mentions nightclub
4,Ultrabar,3.3,$$,414,Night club,911 F St NW · Near Ford's Theatre,Closed ⋅ Opens 10PM Fri,Their website mentions nightclub
5,Heist Night Club,3.7,$$,104,Night club,1802 Jefferson Pl NW,Closed ⋅ Opens 10PM Tue,Luxe bank heist-themed lounge for unique craft...
6,U Street Music Hall,4.3,$,368,Night club,1115A U St NW,Closed ⋅ Opens 7PM,Their website mentions nightclub
7,Eighteenth Street Lounge,4.4,$$,"1,173",Lounge,1212 18th St NW,Closed ⋅ Opens 5PM Tue,"""I highly recommend this place for nightlife. ..."
8,Eden Lounge DC,2.7,$$,152,Night club,1716 I St NW,Closed ⋅ Opens 10PM,Their website mentions nightclub
9,Club Timehri,4.5,$,87,Night club,2439 18th St NW,Closed ⋅ Opens 9PM,"Compact, boisterous joint featuring special ev..."


In [220]:
s = ''
results = driver.find_elements_by_class_name('cXedhc.uQ4NLd')
for d in results:
    #z = d.find_element_by_class_name('dbg0pd')
    d.click()
    time.sleep(5)
    try:
        print(driver.find_element_by_class_name('ggV7z').get_attribute('innerHTML').replace('&amp;', "&").replace('<span>', '').replace('</span>', ''))
    except Exception:
        print(None)
    try:
        desc = d.find_elements_by_class_name('wKLfId')
        for f in desc:
            s += f.get_attribute('innerHTML').replace("&nbsp;·&nbsp;",' · ')
        print(s)
    except Exception:
        pass

Tri-level hipster hangout with snug basement bar, music stage with dance parties & rooftop deck.

Sprawling dance club featuring live hip-hop, rap & R&B in a sleek space with multiple dance floors.

Retro-themed, 3-story nightclub featuring '80s & '90s music amid throwback decor & artwork.

Spacious club with bottle service, high-tech sound & DJs spinning hip-hop, house & Latin music.

Bottle service is available at this multilevel bar & lounge with special lighting effects & DJs.

Luxe bank heist-themed lounge for unique craft cocktails, wines by the glass & top-shelf champagne.

None

Converted townhouse serves up cocktails amid antique sofas & candlelight or on the outdoor patio.

Sprawling, neon-lit club featuring a rooftop deck & 3 themed levels where DJs spin different beats.

Compact, boisterous joint featuring special events with reggae, dancehall, soca & other music.

None

International tunes & American bites at a small dance club with a photo booth & edgy design.

Posh, neon

In [222]:
driver.quit()